In [1]:
from os import listdir
from datetime import timedelta
from warnings import simplefilter

import numpy as np
import pandas as pd

simplefilter("ignore")

# Concatenar Arquivos 2024-01

In [8]:
df_list = []
for file in listdir('./Coletas/Excaulebur/2024-1'):
    df = pd.read_csv(f'./Coletas/Excaulebur/2024-1/{file}', encoding='unicode_escape', engine='python')
    file = file.replace('-A', '').replace('-B', '')
    df['Data'] = pd.to_datetime(file.split('.')[0][-10:], format='%Y-%m-%d', yearfirst=True)
    df['Data'] = df[['Data', 'Hora']].apply(
        lambda x: x[0] + pd.DateOffset(1) if str(x[1]) >= str('00:00:00') 
        and str(x[1]) < str('06:00:00') else x[0], axis=1
    )
    df_list.append(df)
df = pd.concat(df_list, ignore_index=True).sort_values(by=['Data', 'Hora'])
df['Temperatura'] = df['Temperatura'].apply(lambda x: round(float(x[:4]) * 0.01, 3))
df['Umidade'] = df['Umidade'].apply(lambda x: round(int(x[:2]) * 0.01, 2))
df['DataHora'] = pd.to_datetime(df['Data'].dt.strftime('%Y-%m-%d') + ' ' + df['Hora'])
df.drop(['Data', 'Hora'], axis=1, inplace=True)
df.to_csv('./Coletas/Excaulebur/Excaulebur_2024-01.csv', index=False)

In [9]:
df_list = []
for file in listdir('./Coletas/Totosa/2024-1'):
    if 'testA' in file:
        continue
    df = pd.read_csv(f'./Coletas/Totosa/2024-1/{file}', encoding='unicode_escape', engine='python')
    df['Data'] = pd.to_datetime(file.split('.')[0][-10:], format='%Y-%m-%d', yearfirst=True)
    df['Data'] = df[['Data', 'Hora']].apply(
        lambda x: x[0] + pd.DateOffset(1) if str(x[1]) >= str('00:00:00') 
        and str(x[1]) < str('06:00:00') else x[0], axis=1
    )
    df_list.append(df)
df = pd.concat(df_list, ignore_index=True).sort_values(by=['Data', 'Hora'])
df['Temperatura'] = df['Temperatura'].apply(lambda x: round(float(x[:4]) * 0.01, 3))
df['Umidade'] = df['Umidade'].apply(lambda x: round(int(x[:2]) * 0.01, 2))
df['DataHora'] = pd.to_datetime(df['Data'].dt.strftime('%Y-%m-%d') + ' ' + df['Hora'])
df.drop(['Data', 'Hora'], axis=1, inplace=True)
df.to_csv('./Coletas/Totosa/Totosa_2024-01.csv', index=False)

# Concatenar Arquivos 2024-02

In [10]:
df_list = []
for path in listdir('./sensor/'):
    for file in listdir(f'./sensor/{path}/'):
        if '_mes_09' in file:
            continue
        df = pd.read_excel(f'./sensor/{path}/{file}', engine='openpyxl')
        df['Start'] = pd.to_datetime(df['DateTime'])
        df.drop(['DateTime'], axis=1, inplace=True)
        df_list.append(df)
temp_df = pd.concat(df_list, ignore_index=True)
temp_df.columns = ['Temperatura_Sensor', 'Start', 'Umidade_Sensor']
temp_df = temp_df.groupby('Start')[['Temperatura_Sensor', 'Umidade_Sensor']].first()
temp_df.reset_index(inplace=True)
temp_df['Temperatura_Sensor'] = temp_df['Temperatura_Sensor'].apply(lambda x: round(x * 0.01, 3))
temp_df['Umidade_Sensor'] = temp_df['Umidade_Sensor'].apply(lambda x: round(x * 0.01, 2))
temp_df.to_csv('./sensor/Sensor.csv', index=False)
temp_df['End'] = temp_df['Start'] + timedelta(hours=1)

In [11]:
df_list = []
for file in listdir('./Coletas/Excaulebur/2024-2'):
    if file.endswith('.csv'):
        df = pd.read_csv(f'./Coletas/Excaulebur/2024-2/{file}', encoding='unicode_escape', engine='python')
        df['Temperatura'] = np.nan
        df['Umidade'] = np.nan
    elif file.endswith('.xlsx'):
        df = pd.read_excel(f'./Coletas/Excaulebur/2024-2/{file}', engine='openpyxl')
        df['Temperatura'] = df['Temperatura'].apply(lambda x: x if type(x) == float else round(float(x.replace(',', '.').replace('°C', '')) * 0.01, 3))
        df['Dia'] = df['Dia'].dt.strftime('%Y-%m-%d')
    df_list.append(df)
plant_df = pd.concat(df_list, ignore_index=True)
plant_df['DataHora'] = pd.to_datetime(plant_df['Dia'] + ' ' + plant_df['Hora'])
plant_df.drop(['Dia', 'Hora'], axis=1, inplace=True)
plant_df.drop_duplicates(ignore_index=True, inplace=True)

In [12]:
dt = plant_df.DataHora.values
sdt = temp_df.Start.values
edt = temp_df.End.values

i, j = np.where((dt[:, None] >= sdt) & (dt[:, None] < edt))

plant_df = pd.concat([
    plant_df.loc[i, :].reset_index(drop=True),
    temp_df.loc[j, :].reset_index(drop=True)
], axis=1)

plant_df['Temperatura'] = plant_df['Temperatura'].fillna(plant_df['Temperatura_Sensor'])
plant_df['Umidade'] = plant_df['Umidade'].fillna(plant_df['Umidade_Sensor'])
plant_df = plant_df[['Valor do sinal', 'Temperatura', 'Umidade', 'DataHora']]
plant_df.to_csv('./Coletas/Excaulebur/Excaulebur_2024-02.csv', index=False)

In [13]:
df_list = []
for file in listdir('./Coletas/Totosa/2024-2'):
    if file.endswith('.xlsx'):
        df = pd.read_excel(f'./Coletas/Totosa/2024-2/{file}', engine='openpyxl')
        df['Temperatura'] = df['Temperatura'].apply(lambda x: x if type(x) == float else round(float(x.replace(',', '.').replace('°C', '')) * 0.01, 3))
        df['Dia'] = df['Dia'].dt.strftime('%Y-%m-%d')
    df_list.append(df)
plant_df = pd.concat(df_list, ignore_index=True)
plant_df['DataHora'] = pd.to_datetime(plant_df['Dia'] + ' ' + plant_df['Hora'])
plant_df.drop(['Dia', 'Hora'], axis=1, inplace=True)
plant_df.drop_duplicates(ignore_index=True, inplace=True)

In [14]:
dt = plant_df.DataHora.values
sdt = temp_df.Start.values
edt = temp_df.End.values

i, j = np.where((dt[:, None] >= sdt) & (dt[:, None] < edt))

plant_df = pd.concat([
    plant_df.loc[i, :].reset_index(drop=True),
    temp_df.loc[j, :].reset_index(drop=True)
], axis=1)

plant_df['Temperatura'] = plant_df['Temperatura'].fillna(plant_df['Temperatura_Sensor'])
plant_df['Umidade'] = plant_df['Umidade'].fillna(plant_df['Umidade_Sensor'])
plant_df = plant_df[['Valor do sinal', 'Temperatura', 'Umidade', 'DataHora']]
plant_df.to_csv('./Coletas/Totosa/Totosa_2024-02.csv', index=False)

In [2]:
df = pd.concat(pd.read_csv(f'./Coletas/Excaulebur/{file}') for file in listdir('./Coletas/Excaulebur/') if file.endswith('.csv'))
df.drop_duplicates(ignore_index=True, inplace=True)
df.to_csv('./Coletas/Excaulebur/Excaulebur.csv', index=False)

In [3]:
df = pd.concat(pd.read_csv(f'./Coletas/Totosa/{file}') for file in listdir('./Coletas/Totosa/') if file.endswith('.csv'))
df.drop_duplicates(ignore_index=True, inplace=True)
df.to_csv('./Coletas/Totosa/Totosa.csv', index=False)